In [47]:
# pip install -qU langchain-text-splitters
# pip install qdrant-client
# pip install pypdf
# pip install python-dotenv
# pip install sentence-transformers
# pip install qdrant-client

import os
import torch

from dotenv import load_dotenv
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from litellm import completion

load_dotenv()

FILE_PATH = os.path.join("data","12FactorApp.pdf")
EMBEDDING_MODEL = "BAAI/bge-small-en-v1.5"
COLLECTION_NAME = "qa_index"

LLM_MODEL = "ollama/llama2"
API_BASE = "http://localhost:11434"


SYSTEM_PROMPT = """You are an assistant for question-answering tasks. Answer the question according only to the given context.
If question cannot be answered using the context, simply say I don't know. Do not make stuff up.
Context: {context}

"""

USER_PROMPT = """
Question: {question}

Answer:"""


In [48]:
reader = PdfReader(FILE_PATH)
number_of_pages = len(reader.pages)

entire_text = ""
for page_num in range(number_of_pages):
    page = reader.pages[page_num]
    entire_text += page.extract_text()

entire_text[:200]

print(f"Number of pages: {number_of_pages}")

Number of pages: 4


In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)

text_chunks = text_splitter.split_text(entire_text)
print(f"Total chunks: {len(text_chunks)}")

text_chunks[:2]

Total chunks: 28


['The 12-Factor App Methodology\nIn modern software development, applications are often provided as services, such as\nweb apps or software-as-a-service (SaaS). The 12-Factor App methodology offers a\nframework for building such services that:\nThe Twelve Factors',
 'framework for building such services that:\nThe Twelve Factors\nHere is an overview of the twelve factors and their one-sentence definitions:\nDeep Dive into Each Factor\nI. Codebase\nA single codebase should be tracked in a version control system like Git. This codebase']

In [50]:
# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

embedding_model = SentenceTransformer(EMBEDDING_MODEL, device=device)

embeddings = embedding_model.encode(text_chunks, show_progress_bar=True)

embeddings[0].shape

Using device: cpu


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(384,)

docker pull qdrant/qdrant

docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant

In [51]:
client = QdrantClient("http://localhost:6333")

client.delete_collection(COLLECTION_NAME)

client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),

)

True

In [52]:
ids = []
payload = []

for id, text in enumerate(text_chunks):
    ids.append(id)
    payload.append({"source": FILE_PATH, "content": text})

payload[0]

{'source': 'data/12FactorApp.pdf',
 'content': 'The 12-Factor App Methodology\nIn modern software development, applications are often provided as services, such as\nweb apps or software-as-a-service (SaaS). The 12-Factor App methodology offers a\nframework for building such services that:\nThe Twelve Factors'}

In [53]:
client.upload_collection(
    collection_name=COLLECTION_NAME,
    vectors=embeddings,
    payload=payload,
    ids=ids,
    batch_size=256,
)

client.count(COLLECTION_NAME)

CountResult(count=28)

In [54]:
def search(text: str, top_k: int):
    query_embedding = embedding_model.encode(text).tolist()

    search_result = client.search(
        collection_name=COLLECTION_NAME,
        query_vector=query_embedding,
        query_filter=None,
        limit=top_k
    )
    return search_result

In [55]:
question = "Which are the 12 Factors?"
results = search(question, top_k=5)
results

/var/folders/lw/sf857rrx2pnckk7268njzx940000gn/T/ipykernel_77849/566791645.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = client.search(


[ScoredPoint(id=0, version=0, score=0.71610916, payload={'source': 'data/12FactorApp.pdf', 'content': 'The 12-Factor App Methodology\nIn modern software development, applications are often provided as services, such as\nweb apps or software-as-a-service (SaaS). The 12-Factor App methodology offers a\nframework for building such services that:\nThe Twelve Factors'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=1, version=0, score=0.71463484, payload={'source': 'data/12FactorApp.pdf', 'content': 'framework for building such services that:\nThe Twelve Factors\nHere is an overview of the twelve factors and their one-sentence definitions:\nDeep Dive into Each Factor\nI. Codebase\nA single codebase should be tracked in a version control system like Git. This codebase'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=25, version=0, score=0.7134169, payload={'source': 'data/12FactorApp.pdf', 'content': 'Conclusion\nThe 12-Factor App methodology provides a cl

In [56]:
references = [obj.payload["content"] for obj in results]

context = "\n\n".join(references)

In [58]:
response = completion(
    model=LLM_MODEL,
    messages=[{"content": SYSTEM_PROMPT.format(context=context),"role": "system"}, {"content": USER_PROMPT.format(question=question),"role": "user"}],
    api_base=API_BASE,
    stream=True
)

for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")

The 12 factors are:

1. Codebase - A single codebase should be tracked in a version control system like Git.
2. Deployments - Deploying code changes quickly (shortening the time gap).
3. Environments - Creating isolated environments for different deployments of the application.
4. Packaging - Using declarative formats to minimize onboarding time and cost for new developers.
5. Services - Providing services such as web apps or software-as-a-service (SaaS).
6. Config - Managing configuration separate from code.
7. Database - Separating the database into a single, standalone repository.
8. Secret - Storing sensitive information securely.
9. Tests - Writing tests that cover both functional and acceptance testing.
10. Cache - Using caching mechanisms to improve performance.
11. Logs - Centralizing log management for better visibility and troubleshooting.
12. Continuous Integration/Continuous Deployment (CI/CD) - Automating the build, test, and deployment process.